# DM/DM_sen

In [32]:
import pandas as pd
import numpy as np
import pickle
import scipy.optimize as optim
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction
from __future__ import division
import os,sys
import numpy as np
import pandas as pd
from collections import defaultdict
from random import seed, shuffle
from collections import defaultdict
from copy import deepcopy
import numpy.core.multiarray
import cvxpy as cvx
import dccp
from dccp.problem import is_dccp
import utils as ut
import traceback
import random
from sklearn import preprocessing
import time

In [33]:
df = pd.read_csv('../data/compas-scores-two-years.csv')

In [34]:
df.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

In [35]:
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


# Data preprocessing

https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb

**We filtered the data based on criterion provided in 
<br>https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb**

"If the charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested, we assume that because of data quality reasons, that we do not have the right offense.
<br>We coded the recidivist flag -- is_recid -- to be -1 if we could not find a compas case at all.
<br>In a similar vein, ordinary traffic offenses -- those with a c_charge_degree of 'O' -- will not result in Jail time are removed (only two of them).
<br>We filtered the underlying data from Broward county to include only those rows representing people who had either recidivated in two years, or had at least two years outside of a correctional facility."

In [36]:
df = df.loc[(df["days_b_screening_arrest"]<=30) & (df["days_b_screening_arrest"]>=-30)]
df = df.loc[df["is_recid"] != -1]
df = df.loc[df["c_charge_degree"] != "O"]
df = df.loc[(df["race"] == "African-American") | (df["race"] == "Caucasian")]

In [37]:
# Set two_year_recid as y convert the negative label from 0 to -1 for classifiers
y = df['two_year_recid'].to_numpy()
y[y==0] = -1
len_rows = len(y)

In [38]:
features = ["race", "sex", "age_cat", "c_charge_degree", "priors_count", "juv_fel_count"] # all features
cont_features = ["priors_count", "juv_fel_count"] # continuous features 
sensitive_attrs = ["race"]

In [39]:
X_control = np.array([]).reshape(len_rows,0)
X = np.array([]).reshape(len_rows, 0) 
feature_names = []

for feature in features:
    vals = df[feature]
    if feature in cont_features:
        # normalize
        vals = preprocessing.scale(vals)
        # convert to 2d array
        vals  = np.reshape(vals, (len_rows, -1))
    else:
        # use binary labels
        lb = preprocessing.LabelBinarizer()
        lb.fit(vals)
        vals = lb.transform(vals)
    
    if feature in sensitive_attr:
        X_control = np.hstack((X_control, vals))
    
    X = np.hstack((X, vals))
    if vals.shape[1] == 1:
        feature_names.append(feature)
    else:
        for c in lb.classes_:
            feature_names.append(f"{feature}_{c}")

In [40]:
# add intercept to data
m,n = X.shape
intercept = np.ones(m).reshape(m, 1)
X = np.concatenate((intercept,X), axis = 1)

feature_names = ["intercept"] + feature_names
assert(len(feature_names) == X.shape[1])

In [41]:
def split_into_train_test(x_all, y_all, x_control_all, train_fold_size):
    
    split_point = int(round(float(x_all.shape[0]) * train_fold_size))
    x_all_train = x_all[:split_point]
    x_all_test = x_all[split_point:]
    y_all_train = y_all[:split_point]
    y_all_test = y_all[split_point:]
    x_control_all_train = {}
    x_control_all_test = {}
    #for k in x_control_all.keys():
    x_control_all_train = x_control_all[:split_point]
    x_control_all_test = x_control_all[split_point:]
    normalizer = preprocessing.Normalizer()
    x_all_train = normalizer.fit_transform(x_all_train)

    return x_all_train, y_all_train, {"race": x_control_all_train.flatten()}, x_all_test, y_all_test, {"race":x_control_all_test.flatten()}

train_fold_size = 0.5
X_train, y_train, X_control_train, X_test, y_test, X_control_test = split_into_train_test(X, y, X_control, train_fold_size)

# Objective functions with constraints

The following formulas are summarized in paper https://arxiv.org/abs/1610.08452.
<br>In this section, we implement methods provided in the paper's author's github repo https://github.com/mbilalzafar/fair-classification/blob/master/fair_classification/utils.py that use OMR, FPR, FNR, and both FNR and FPR as constraints in our logistic regressions and SVM.

<img src='../figs/dm_notions.png' width=500>

In [42]:
def train_model_disp_mist(X, y, X_control, loss_function, EPS, cons_params):

    # cons_type, sensitive_attrs_to_cov_thresh, take_initial_sol, gamma, tau, mu, EPS, cons_type
    """
    Function that trains the model subject to various fairness constraints.
    If no constraints are given, then simply trains an unaltered classifier.
    Example usage in: "disparate_mistreatment/synthetic_data_demo/decision_boundary_demo.py"
    ----
    Inputs:
    X: (n) x (d+1) numpy array -- n = number of examples, d = number of features, one feature is the intercept
    y: 1-d numpy array (n entries)
    x_control: dictionary of the type {"s": [...]}, key "s" is the sensitive feature name, and the value is a 1-d list with n elements holding the sensitive feature values
    loss_function: the loss function that we want to optimize -- for now we have implementation of logistic loss, but other functions like hinge loss can also be added
    EPS: stopping criteria for the convex solver. check the CVXPY documentation for details. default for CVXPY is 1e-6
    cons_params: is None when we do not want to apply any constraints
    otherwise: cons_params is a dict with keys as follows:
        - cons_type: 
            - 0 for all misclassifications 
            - 1 for FPR
            - 2 for FNR
            - 4 for both FPR and FNR
        - tau: DCCP parameter, controls how much weight to put on the constraints, if the constraints are not satisfied, then increase tau -- default is DCCP val 0.005
        - mu: DCCP parameter, controls the multiplicative factor by which the tau increases in each DCCP iteration -- default is the DCCP val 1.2
        - take_initial_sol: whether the starting point for DCCP should be the solution for the original (unconstrained) classifier -- default value is True
        - sensitive_attrs_to_cov_thresh: covariance threshold for each cons_type, eg, key 1 contains the FPR covariance
    ----
    Outputs:
    w: the learned weight vector for the classifier
    """

    max_iters = 100 # for the convex program
    max_iter_dccp = 50  # for the dccp algo

    
    num_points, num_features = X.shape
    w = cvx.Variable(num_features) # this is the weight vector

    # initialize a random value of w
    np.random.seed(5243)
    w.value = np.random.rand(X.shape[1])

    if cons_params is None: # just train a simple classifier, no fairness constraints
        constraints = []
    else:
        constraints = get_constraint_list_cov(X, y, X_control, cons_params["sensitive_attrs_to_cov_thresh"], cons_params["cons_type"], w)


    if loss_function == "logreg":
        # constructing the logistic loss problem
        loss = cvx.sum( cvx.logistic( cvx.multiply(-y, X*w) )  ) / num_points # we are converting y to a diagonal matrix for consistent
        #loss = cvx.sum(cvx.multiply(y, x*w) - cvx.logistic(x*w))
    elif loss_function == "svm":
        loss = cvx.sum( cvx.pos(1 - cvx.multiply(y, X*w))) / num_points
        
    # sometimes, its a good idea to give a starting point to the constrained solver
    # this starting point for us is the solution to the unconstrained optimization problem
    # another option of starting point could be any feasible solution
    if cons_params is not None:
        if cons_params.get("take_initial_sol") is None: # true by default
            take_initial_sol = True
        elif cons_params["take_initial_sol"] == False:
            take_initial_sol = False

        if take_initial_sol == True: # get the initial solution
            p = cvx.Problem(cvx.Minimize(loss), [])
            p.solve()


    # construct the cvxpy problem
    prob = cvx.Problem(cvx.Minimize(loss), constraints)

    # print "\n\n"
    # print "Problem is DCP (disciplined convex program):", prob.is_dcp()
    # print "Problem is DCCP (disciplined convex-concave program):", is_dccp(prob)

    try:

        tau, mu = 0.005, 1.2 # default dccp parameters, need to be varied per dataset
        if cons_params is not None: # in case we passed these parameters as a part of dccp constraints
            if cons_params.get("tau") is not None: tau = cons_params["tau"]
            if cons_params.get("mu") is not None: mu = cons_params["mu"]

        prob.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,
            solver='ECOS', verbose=False, 
            feastol=EPS, abstol=EPS, reltol=EPS,feastol_inacc=EPS, abstol_inacc=EPS, reltol_inacc=EPS,
            max_iters=max_iters, max_iter=max_iter_dccp)

        
        assert(prob.status == "Converged" or prob.status == "optimal")
        # print "Optimization done, problem status:", prob.status

    except:
        traceback.print_exc()
        sys.stdout.flush()
        sys.exit(1)


    # check that the fairness constraint is satisfied
    for f_c in constraints:
       # assert(f_c.value == True) # can comment this out if the solver fails too often, but make sure that the constraints are satisfied empirically. alternatively, consider increasing tau parameter
        pass
        

    w = np.array(w.value).flatten() # flatten converts it to a 1d array


    return w

In [43]:
def get_constraint_list_cov(X_train, y_train, X_control_train, sensitive_attrs_to_cov_thresh, cons_type, w):

    """
    get the list of constraints to be fed to the minimizer
    cons_type == 0: means the whole combined misclassification constraint (without FNR or FPR)
    cons_type == 1: FPR constraint
    cons_type == 2: FNR constraint
    cons_type == 4: both FPR as well as FNR constraints
    sensitive_attrs_to_cov_thresh: is a dict like {s: {cov_type: val}}
    s is the sensitive attr
    cov_type is the covariance type. contains the covariance for all misclassifications, FPR and for FNR etc
    """

    constraints = []
    for attr in sensitive_attrs_to_cov_thresh.keys():
        attr_arr = X_control_train[attr]
        #attr_arr_transformed, index_dict = get_one_hot_encoding(attr_arr)
        index_dict = None        
        if index_dict is None: # binary attribute, in this case, the attr_arr_transformed is the same as the attr_arr

            s_val_to_total = {ct:{} for ct in [0,1,2]} # constrain type -> sens_attr_val -> total number
            s_val_to_avg = {ct:{} for ct in [0,1,2]}
            cons_sum_dict = {ct:{} for ct in [0,1,2]} # sum of entities (females and males) in constraints are stored here

            for v in set(attr_arr):
                s_val_to_total[0][v] = sum(X_control_train[attr] == v)
                s_val_to_total[1][v] = sum(np.logical_and(X_control_train[attr] == v, y_train == -1)) # FPR constraint so we only consider the ground truth negative dataset for computing the covariance
                s_val_to_total[2][v] = sum(np.logical_and(X_control_train[attr] == v, y_train == +1))


            for ct in [0,1,2]:
                s_val_to_avg[ct][0] = s_val_to_total[ct][1] / float(s_val_to_total[ct][0] + s_val_to_total[ct][1]) # N1/N in our formulation, differs from one constraint type to another
                s_val_to_avg[ct][1] = 1.0 - s_val_to_avg[ct][0] # N0/N

            
            for v in set(attr_arr):

                idx = X_control_train[attr] == v                


                #################################################################
                # #DCCP constraints
                dist_bound_prod = cvx.multiply(y_train[idx], X_train[idx] * w) # y.f(x)
                
                cons_sum_dict[0][v] = cvx.sum( cvx.minimum(0, dist_bound_prod) ) * (s_val_to_avg[0][v] / len(X_train)) # avg misclassification distance from boundary
                cons_sum_dict[1][v] = cvx.sum( cvx.minimum(0, cvx.multiply( (1 - y_train[idx])/2.0, dist_bound_prod) ) ) * (s_val_to_avg[1][v] / sum(y_train == -1)) # avg false positive distance from boundary (only operates on the ground truth neg dataset)
                cons_sum_dict[2][v] = cvx.sum( cvx.minimum(0, cvx.multiply( (1 + y_train[idx])/2.0, dist_bound_prod) ) ) * (s_val_to_avg[2][v] / sum(y_train == +1)) # avg false negative distance from boundary
                #################################################################

                
            if cons_type == 4:
                cts = [1,2]
            elif cons_type in [0,1,2]:
                cts = [cons_type]
            
            else:
                raise Exception("Invalid constraint type")


            #################################################################
            #DCCP constraints
            for ct in cts:
                thresh = abs(sensitive_attrs_to_cov_thresh[attr][ct][1] - sensitive_attrs_to_cov_thresh[attr][ct][0])
                constraints.append( cons_sum_dict[ct][1] <= cons_sum_dict[ct][0]  + thresh )
                constraints.append( cons_sum_dict[ct][1] >= cons_sum_dict[ct][0]  - thresh )

            #################################################################


            
        else: # otherwise, its a categorical attribute, so we need to set the cov thresh for each value separately
            # need to fill up this part
            raise Exception("Fill the constraint code for categorical sensitive features... Exiting...")
            sys.exit(1)
            

    return constraints

# Methods for evaluation

In [44]:
def return_accuracy_1():
    d0 = y-predicted_labels
    d1 = d0[d0 != 0]
    all_pro = len(d1)/len(y)
    # FPR y = -1, prediction = 1
    d2 = d0[d0 == -2]
    y2 = y[y == -1]
    fpr_pro = len(d2)/len(y2)
    # FNR y = 1, prediction = -1
    d3 = d0[d0 == 2]
    y3 = y[y == 1]
    fnr_pro = len(d3)/len(y3)
    print("ALL:"+str(all_pro),"FPR:"+str(fpr_pro),"FNR:"+str(fnr_pro))

In [45]:
def get_fpr_fnr_sensitive_features(y_true, y_pred, x_control, sensitive_attrs, verbose = False):



    # we will make some changes to x_control in this function, so make a copy in order to preserve the origianl referenced object
    x_control_internal = deepcopy(x_control)

    s_attr_to_fp_fn = {}
    
    for s in sensitive_attrs:
        s_attr_to_fp_fn[s] = {}
        s_attr_vals = x_control_internal[s]
        if verbose == True:
            print( "||  s  || FPR. || FNR. ||")
        for s_val in sorted(list(set(s_attr_vals))):
            s_attr_to_fp_fn[s][s_val] = {}
            y_true_local = y_true[s_attr_vals==s_val]
            y_pred_local = y_pred[s_attr_vals==s_val]

            

            acc = float(sum(y_true_local==y_pred_local)) / len(y_true_local)

            fp = sum(np.logical_and(y_true_local == -1.0, y_pred_local == +1.0)) # something which is -ve but is misclassified as +ve
            fn = sum(np.logical_and(y_true_local == +1.0, y_pred_local == -1.0)) # something which is +ve but is misclassified as -ve
            tp = sum(np.logical_and(y_true_local == +1.0, y_pred_local == +1.0)) # something which is +ve AND is correctly classified as +ve
            tn = sum(np.logical_and(y_true_local == -1.0, y_pred_local == -1.0)) # something which is -ve AND is correctly classified as -ve

            all_neg = sum(y_true_local == -1.0)
            all_pos = sum(y_true_local == +1.0)

            fpr = float(fp) / float(fp + tn)
            fnr = float(fn) / float(fn + tp)
            tpr = float(tp) / float(tp + fn)
            tnr = float(tn) / float(tn + fp)


            s_attr_to_fp_fn[s][s_val]["fp"] = fp
            s_attr_to_fp_fn[s][s_val]["fn"] = fn
            s_attr_to_fp_fn[s][s_val]["fpr"] = fpr
            s_attr_to_fp_fn[s][s_val]["fnr"] = fnr

            s_attr_to_fp_fn[s][s_val]["acc"] = (tp + tn) / (tp + tn + fp + fn)
            if verbose == True:
                if isinstance(s_val, float): # print the int value of the sensitive attr val
                    s_val = int(s_val)
                print ("||  %s  || %0.2f || %0.2f ||" % (s_val, fpr, fnr))

        
        return s_attr_to_fp_fn

In [46]:
def get_sensitive_attr_constraint_fpr_fnr_cov(model, X_arr, y_arr_true, y_arr_dist_boundary, X_control_arr, verbose=False):

    
    """
    Here we compute the covariance between sensitive attr val and ONLY misclassification distances from boundary for False-positives
    (-N_1 / N) sum_0(min(0, y.f(x))) + (N_0 / N) sum_1(min(0, y.f(x))) for all misclassifications
    (-N_1 / N) sum_0(min(0, (1-y)/2 . y.f(x))) + (N_0 / N) sum_1(min(0,  (1-y)/2. y.f(x))) for FPR
    y_arr_true are the true class labels
    y_arr_dist_boundary are the predicted distances from the decision boundary
    If the model is None, we assume that the y_arr_dist_boundary contains the distace from the decision boundary
    If the model is not None, we just compute a dot product or model and x_arr
    for the case of SVM, we pass the distace from bounday becase the intercept in internalized for the class
    and we have compute the distance using the project function
    this function will return -1 if the constraint specified by thresh parameter is not satifsified
    otherwise it will reutrn +1
    if the return value is >=0, then the constraint is satisfied
    """

        
    assert(X_arr.shape[0] == X_control_arr.shape[0])
    if len(X_control_arr.shape) > 1: # make sure we just have one column in the array
        assert(X_control_arr.shape[1] == 1)
    if len(set(X_control_arr)) != 2: # non binary attr
        raise Exception("Non binary attr, fix to handle non bin attrs")

    
    arr = []
    if model is None:
        arr = y_arr_dist_boundary * y_arr_true # simply the output labels
    else:
        arr = np.dot(model, X_arr.T) * y_arr_true # the product with the weight vector -- the sign of this is the output label
    arr = np.array(arr)

    s_val_to_total = {ct:{} for ct in [0,1,2]}
    s_val_to_avg = {ct:{} for ct in [0,1,2]}
    cons_sum_dict = {ct:{} for ct in [0,1,2]} # sum of entities (females and males) in constraints are stored here

    for v in set(X_control_arr):
        s_val_to_total[0][v] = sum(X_control_arr == v)
        s_val_to_total[1][v] = sum(np.logical_and(X_control_arr == v, y_arr_true == -1))
        s_val_to_total[2][v] = sum(np.logical_and(X_control_arr == v, y_arr_true == +1))


    for ct in [0,1,2]:
        s_val_to_avg[ct][0] = s_val_to_total[ct][1] / float(s_val_to_total[ct][0] + s_val_to_total[ct][1]) # N1 / N
        s_val_to_avg[ct][1] = 1.0 - s_val_to_avg[ct][0] # N0 / N

    
    for v in set(X_control_arr):
        idx = X_control_arr == v
        dist_bound_prod = arr[idx]

        cons_sum_dict[0][v] = sum( np.minimum(0, dist_bound_prod) ) * (s_val_to_avg[0][v] / len(X_arr))
        cons_sum_dict[1][v] = sum( np.minimum(0, ( (1 - y_arr_true[idx]) / 2 ) * dist_bound_prod) ) * (s_val_to_avg[1][v] / sum(y_arr_true == -1))
        cons_sum_dict[2][v] = sum( np.minimum(0, ( (1 + y_arr_true[idx]) / 2 ) * dist_bound_prod) ) * (s_val_to_avg[2][v] / sum(y_arr_true == +1))
        

    cons_type_to_name = {0:"ALL", 1:"FPR", 2:"FNR"}
    for cons_type in [0,1,2]:
        cov_type_name = cons_type_to_name[cons_type]    
        cov = cons_sum_dict[cons_type][1] - cons_sum_dict[cons_type][0]
        if verbose == True:
            print( "Covariance for type '%s' is: %0.7f" %(cov_type_name, cov))
        
    return cons_sum_dict

In [47]:
def check_accuracy(model, X_train, y_train, X_test, y_test, y_train_predicted, y_test_predicted):


    """
    returns the train/test accuracy of the model
    we either pass the model (w)
    else we pass y_predicted
    """
    if model is not None and y_test_predicted is not None:
        print ("Either the model (w) or the predicted labels should be None")
        raise Exception("Either the model (w) or the predicted labels should be None")

    if model is not None:
        y_test_predicted = np.sign(np.dot(X_test, model))
        y_train_predicted = np.sign(np.dot(X_train, model))

    def get_accuracy(y, Y_predicted):
        correct_answers = (Y_predicted == y).astype(int) # will have 1 when the prediction and the actual label match
        accuracy = float(sum(correct_answers)) / float(len(correct_answers))
        return accuracy, sum(correct_answers)

    train_score, correct_answers_train = get_accuracy(y_train, y_train_predicted)
    test_score, correct_answers_test = get_accuracy(y_test, y_test_predicted)

    return train_score, test_score, correct_answers_train, correct_answers_test

In [48]:
def get_distance_boundary(w, X, s_attr_arr):

    """
        if we have boundaries per group, then use those separate boundaries for each sensitive group
        else, use the same weight vector for everything
    """

    distances_boundary = np.zeros(X.shape[0])
    if isinstance(w, dict): # if we have separate weight vectors per group
        for k in w.keys(): # for each w corresponding to each sensitive group
            d = np.dot(X, w[k])
            distances_boundary[s_attr_arr == k] = d[s_attr_arr == k] # set this distance only for people with this sensitive attr val
    else: # we just learn one w for everyone else
        distances_boundary = np.dot(X, w)
    return distances_boundary

In [49]:
def get_clf_stats(w, X_train, y_train, X_control_train, X_test, y_test, x_control_test, sensitive_attrs):



    
    assert(len(sensitive_attrs) == 1) # ensure that we have just one sensitive attribute
    s_attr = "race" # for now, lets compute the accuracy for just one sensitive attr
    #s_attr = sensitive_attrs[0] # for now, lets compute the accuracy for just one sensitive attr


    # compute distance from boundary
    distances_boundary_train = get_distance_boundary(w, X_train, X_control_train[s_attr])
    distances_boundary_test = get_distance_boundary(w, X_test, X_control_test[s_attr])

    # compute the class labels
    all_class_labels_assigned_train = np.sign(distances_boundary_train)
    all_class_labels_assigned_test = np.sign(distances_boundary_test)


    train_score, test_score, correct_answers_train, correct_answers_test = check_accuracy(None, X_train, y_train, X_test, y_test, all_class_labels_assigned_train, all_class_labels_assigned_test)

    
    cov_all_train = {}
    cov_all_test = {}
    for s_attr in sensitive_attrs:
        
        
        print_stats = False # we arent printing the stats for the train set to avoid clutter

        # uncomment these lines to print stats for the train fold
        # print "*** Train ***"
        # print "Accuracy: %0.3f" % (train_score)
        # print_stats = True
        s_attr_to_fp_fn_train = get_fpr_fnr_sensitive_features(y_train, all_class_labels_assigned_train, X_control_train, sensitive_attrs, print_stats)
        cov_all_train[s_attr] = get_sensitive_attr_constraint_fpr_fnr_cov(None, X_train, y_train, distances_boundary_train, X_control_train[s_attr]) 
        

        print ("\n")
        print( "Accuracy: %0.3f" % (test_score))
        print_stats = True # only print stats for the test fold
        s_attr_to_fp_fn_test = get_fpr_fnr_sensitive_features(y_test, all_class_labels_assigned_test, X_control_test, sensitive_attrs, print_stats)
        cov_all_test[s_attr] = get_sensitive_attr_constraint_fpr_fnr_cov(None, X_test, y_test, distances_boundary_test, X_control_test[s_attr]) 
        print ("\n")

    return train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test

In [50]:
def train_test_classifier():
    w = train_model_disp_mist(X_train, y_train, X_control_train, loss_function, EPS, cons_params)

    train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = get_clf_stats(w, X_train, y_train, X_control_train, X_test, y_test, X_control_test, sensitive_attrs)

    # accuracy and FPR are for the test because we need of for plotting
    return w, test_score, s_attr_to_fp_fn_test

In [51]:
def return_accuracy_noConstraint():
    """ Classify the data while optimizing for accuracy """
    print("== Unconstrained (original) classifier ==")
    cons_params['cons_type'] = 0 # FPR constraint -- just change the cons_type, the rest of parameters should stay the same
    print(cons_params)
    tau = 5.0
    mu = 1.2
    sensitive_attrs_to_cov_thresh = {"race": {0:{0:0, 1:0}, 1:{0:0, 1:0}, 2:{0:0, 1:0}}} # zero covariance threshold, means try to get the fairest solution
    w_cons, acc_cons, s_attr_to_fp_fn_test_cons  = train_test_classifier()
    # print("\n-----------------------------------------------------------------------------------\n")

In [52]:
def return_accuracy_FPR():
    """ Now classify such that we optimize for accuracy while achieving perfect fairness """
    print("\n\n== Constraints on FPR ==") # setting parameter for constraints
    cons_params['cons_type'] = 1 # FPR constraint -- just change the cons_type, the rest of parameters should stay the same
    print(cons_params)
    tau = 5.0
    mu = 1.2
    sensitive_attrs_to_cov_thresh = {"race": {0:{0:0, 1:0}, 1:{0:0, 1:0}, 2:{0:0, 1:0}}} # zero covariance threshold, means try to get the fairest solution
    w_cons, acc_cons, s_attr_to_fp_fn_test_cons  = train_test_classifier()

In [53]:
def return_accuracy_FNR():
    """ Now classify such that we optimize for accuracy while achieving perfect fairness """
    print("\n\n== Constraints on FNR ==") # setting parameter for constraints
    cons_params['cons_type'] = 2 # FNR constraint -- just change the cons_type, the rest of parameters should stay the same
    print(cons_params)
    tau = 5.0
    mu = 1.2
    sensitive_attrs_to_cov_thresh = {"race": {0:{0:0, 1:0}, 1:{0:0, 1:0}, 2:{0:0, 1:0}}} # zero covariance threshold, means try to get the fairest solution
    w_cons, acc_cons, s_attr_to_fp_fn_test_cons  = train_test_classifier()

In [54]:
def return_accuracy_allConstraints():
    """ Now classify such that we optimize for accuracy while achieving perfect fairness """
    print("\n\n== Constraints on FNR and FPR ==") # setting parameter for constraints
    cons_params['cons_type'] = 4 # FNR constraint -- just change the cons_type, the rest of parameters should stay the same
    print(cons_params)
    tau = 5.0
    mu = 1.2
    sensitive_attrs_to_cov_thresh = {"race": {0:{0:0, 1:0}, 1:{0:0, 1:0}, 2:{0:0, 1:0}}} # zero covariance threshold, means try to get the fairest solution
    w_cons, acc_cons, s_attr_to_fp_fn_test_cons  = train_test_classifier()

# Evaluation

## Logistic regression

In [55]:
import warnings
warnings.filterwarnings('ignore')

loss_function = "logreg" 
EPS = 1e-6
cons_type = 0 # No constraint at very beginning 
tau = 5.0
mu = 1.2
sensitive_attrs_to_cov_thresh = {"race": {0:{0:0, 1:0}, 1:{0:0, 1:0}, 2:{0:0, 1:0}}} # zero covariance threshold, means try to get the fairest solution

cons_params = None
cons_params = {"cons_type": cons_type, "tau": tau, "mu": mu, "sensitive_attrs_to_cov_thresh": sensitive_attrs_to_cov_thresh}
start_dm = time.time()
return_accuracy_noConstraint()
return_accuracy_FPR()
return_accuracy_FNR()
return_accuracy_allConstraints()

end_dm = time.time()
runtime_dm = (end_dm-start_dm)
print(f'runtime of the complete DM model is {np.round(runtime_dm, 2)} seconds')

== Unconstrained (original) classifier ==
{'cons_type': 0, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.660
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.32 ||
||  1  || 0.18 || 0.62 ||




== Constraints on FPR ==
{'cons_type': 1, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.649
||  s  || FPR. || FNR. ||
||  0  || 0.27 || 0.41 ||
||  1  || 0.25 || 0.53 ||




== Constraints on FNR ==
{'cons_type': 2, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.651
||  s  || FPR. || FNR. ||
||  0  || 0.28 || 0.39 ||
||  1  || 0.29 || 0.47 ||




== Constraints on FNR and FPR ==
{'cons_type': 4, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.651
||  s  || FPR

## Support vector machine (SVM)

In [26]:
import warnings
warnings.filterwarnings('ignore')

loss_function = "svm" 
EPS = 1e-6
cons_type = 0 # No constraint at very beginning 
tau = 5.0
mu = 1.2
sensitive_attrs_to_cov_thresh = {"race": {0:{0:0, 1:0}, 1:{0:0, 1:0}, 2:{0:0, 1:0}}} # zero covariance threshold, means try to get the fairest solution

cons_params = None
cons_params = {"cons_type": cons_type, "tau": tau, "mu": mu, "sensitive_attrs_to_cov_thresh": sensitive_attrs_to_cov_thresh}
start_dm = time.time()
return_accuracy_noConstraint()
return_accuracy_FPR()
return_accuracy_FNR()
return_accuracy_allConstraints()

end_dm = time.time()
runtime_dm = (end_dm-start_dm)
print(f'runtime of the complete DM model is {np.round(runtime_dm, 2)} seconds')

== Unconstrained (original) classifier ==
{'cons_type': 0, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.649
||  s  || FPR. || FNR. ||
||  0  || 0.38 || 0.30 ||
||  1  || 0.20 || 0.62 ||




== Constraints on FPR ==
{'cons_type': 1, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.650
||  s  || FPR. || FNR. ||
||  0  || 0.31 || 0.37 ||
||  1  || 0.27 || 0.51 ||




== Constraints on FNR ==
{'cons_type': 2, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.652
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.35 ||
||  1  || 0.27 || 0.50 ||




== Constraints on FNR and FPR ==
{'cons_type': 4, 'tau': 5.0, 'mu': 1.2, 'sensitive_attrs_to_cov_thresh': {'race': {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0}, 2: {0: 0, 1: 0}}}}


Accuracy: 0.650
||  s  || FPR